In [1]:
import re
import pandas as pd
from csv import writer
from keras.preprocessing.text import text_to_word_sequence

Using TensorFlow backend.


In [2]:
# tags from corpus
tagged_corpus = pd.read_csv("data/corpus_tags.tsv", sep="\t", names=["pmid", "start", "end", "word", "type", "umls_id"])
tagged_corpus = tagged_corpus.dropna()
tagged_corpus.iloc[610:640]

,pmid,start,end,word,type,umls_id
610,26867927,0,23,Radiofrequency ablation,T061,C0850292
611,26867927,27,59,posteroseptal accessory pathways,T023,C0447025
612,26867927,60,75,associated with,T080,C0332281
613,26867927,76,102,coronary sinus diverticula,"T019,T047",C3163894
614,26867927,103,135,Posteroseptal accessory pathways,T023,C0447025
615,26867927,143,158,associated with,T080,C0332281
616,26867927,161,193,coronary sinus (CS) diverticulum,"T019,T047",C3163894
617,26867927,227,251,clinical characteristics,T201,C0683325
618,26867927,253,260,mapping,T052,C1283195
619,26867927,265,291,ablation of these pathways,T061,C3275043


In [3]:
classes = []
with open("data/vocab_class.txt") as ftags:
    classes = [i.strip() for i in ftags.readlines()]
    ftags.close()
print(classes)

['T047', 'T005', 'T007', 'T016']


In [0]:
 # Assisgns each word its respective tag with "O" as the default tag
sentence_count = 1
com_words = ['~']   #completes words when running through corpus

def get_words_tags(text, pmid,start=0):
    words = []
    tags = []  
    row_cont = []
    typ = "O"
    test = True
    global sentence_count
    df = tagged_corpus[(tagged_corpus["pmid"]==int(pmid)) & (tagged_corpus["end"] <= start + len(text)) & (tagged_corpus["start"] >= start)]
    indices = df.index.values
    text_split = text_to_word_sequence(text, filters='!"#$%&()*+,-/:;<=>?@[\\]^_`{|}~\t\n', lower = False)
    for word in text_split:
      for i in indices:
        text2 = df.loc[i]["word"]
        text2_split = text_to_word_sequence(text2, filters='!"#$%&()*+,-/:;<=>?@[\\]^_`{|}~\t\n', lower = False)
        if word in text2_split and set(df.loc[i]["type"].split(',')).intersection(classes):
          typ = ','.join(list(set(df.loc[i]["type"].split(',')).intersection(classes)))
          if not word.isnumeric():
            #print(word,' # ',word in com_words)
            if word.lower() in com_words and test:
              test = True
            else:
              test = False
              com_words.append(word.lower())
          break
        else:
          typ = "O"
      row_cont.append(["Sentence: " + str(sentence_count),word,typ])
    sentence_count = sentence_count + 1
    return row_cont,test



In [0]:
title_regex = re.compile("^\d+\|t\|")
abstract_regex = re.compile("^\d+\|a\|")
train_file = "data/train_" + ''.join(classes) + ".csv"
test_file = "data/test_" + ''.join(classes) + ".csv"

fr = open("data/corpus_pubtator.txt") #corpus file should be placed here
#dataset_file = "dataset_T047.csv" 
f1 = open(train_file,'a')
f1_writer = writer(f1)
first_row = ["Sentence #","Word","Tag"]
f1_writer.writerow(first_row)
f1.close()

f2 = open(test_file,'a')
f2_writer = writer(f2)
first_row = ["Sentence #","Word","Tag"]
f2_writer.writerow(first_row)
f2.close()

In [0]:
def append_list_as_row(file_name, list_of_elem):
    # Open file in append mode
    with open(file_name, 'a', newline='') as write_obj:
        # Create a writer object from csv module
        csv_writer = writer(write_obj)
        # Add contents of list as last row in the csv file
        for row in list_of_elem:
          csv_writer.writerow(row)
        blank = ['','','']
        csv_writer.writerow(blank)

In [0]:
from tqdm.notebook import trange

abstract_count = 0
train_count = 0
test_count = 0
__ = len(fr.readlines())
fr.seek(0)
lines = fr.readlines()
print('Classes being used: ',classes)

for i,_ in zip(lines,trange(__, desc = 'Abstract',ncols='700px')):
    row_content = []
    line = i.strip()
    title_search = title_regex.search(line)
    abstract_search = abstract_regex.search(line)
    if title_search:
        title_line = title_regex.sub('',line)
    elif abstract_search:
        abstract = abstract_regex.sub('',line)
        pmid = int(abstract_search.group(0)[:-3])
        row_content,test = get_words_tags(title_line,pmid,0)
        if test:
          append_list_as_row(test_file, row_content)
          test_count += 1
        else:
           append_list_as_row(train_file, row_content)
           train_count += 1
        title_size = len(title_line) + 1                     # 1 for \n        
        for sentence in re.split("\. ", abstract):
            if len(sentence) == 0:
                continue
            _size = abstract.index(sentence) + title_size
            row_content,test = get_words_tags(sentence, pmid,_size)
            if test and (train_count/(train_count+test_count)) >= 0.70:
              append_list_as_row(test_file, row_content)
              test_count += 1
            else:
              append_list_as_row(train_file, row_content)
              train_count += 1
        abstract_count += 1
        if abstract_count%100 == 0:
            print("PMID {} written to file. Abstract Count: {} Train_count: {} Test_count: {} split: {:.3f}".format(pmid, abstract_count,train_count, test_count, train_count/(train_count+test_count)))
#fr.close()

Classes being used:  ['T047', 'T005', 'T007', 'T016']


PMID 27255600 written to file. Abstract Count: 100 Train_count: 798 Test_count: 343 split: 0.699


In [0]:
print('__     ______    ___    __   _________')
print('| \   /  __  \  | \ \   | |  |   _____|')
print('|  |  | |  | |  | |\ \  | |  |  |_____')
print('|  |  | |__| |  | | \ \ | |  |  |_____')
print('|_/   \______/  |_|  \_\|_|  |________|')

In [0]:
import csv
def conv(inputPath):
  print("Converting CSV to tab-delimited file...")
  outputPath = inputPath[:-4]+".txt"
  with open(inputPath) as inputFile:
	  next(inputFile)
	  with open(outputPath, 'w', newline='') as outputFile:
		  reader = csv.DictReader(inputFile, delimiter=',')
		  writer = csv.DictWriter(outputFile, reader.fieldnames, delimiter='\t')
		  writer.writeheader()
		  writer.writerows(reader)
  print("Conversion complete.", outputPath)

In [0]:
conv(train_file)
conv(test_file)

In [0]:
import re

file_1 = open(train_file[0:-3]+'txt', 'r')
file_2 = open(test_file[0:-3]+'txt', 'r')
sentence_regex = re.compile("^(Sentence: )\d+")
sent_no = 0
sent_count_train = 0
sent_count_test = 0
tmp =0

for line in file_1.readlines():
  if len(line) <=3:
    continue
  sentence_search = sentence_regex.search(line)
  sent_no = int(sentence_search.group(0)[9:])
  if sent_no != tmp:
    sent_count_train += 1
    tmp = sent_no

for line in file_2.readlines():
  if len(line) <=3:
    continue
  sentence_search = sentence_regex.search(line)
  sent_no = int(sentence_search.group(0)[9:])
  if sent_no != tmp:
    sent_count_test += 1
    tmp = sent_no

print(sent_count_train, sent_count_test)
print(sent_count_train/(sent_count_train+sent_count_test)," | ",sent_count_test/(sent_count_train+sent_count_test))